In [83]:
from selenium.webdriver.chrome.service import Service
from selenium import webdriver

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

#beautifulsoup
import requests as req
from bs4 import BeautifulSoup as bs
from pprint import pprint

In [84]:
driver = webdriver.Chrome(
)
driver.get("https://judgment.judicial.gov.tw/FJUD/Default_AD.aspx")

sleep(1)

In [85]:
input_case_reason = driver.find_element(By.CSS_SELECTOR, 'input#jud_jmain')
input_case_reason.send_keys("殺人")
input_case_content = driver.find_element(By.CSS_SELECTOR, 'input#jud_kw')
input_case_content.send_keys("中華民國刑法第 271 條&刑事判決")

sleep(2)

# 送出搜尋
search_button = driver.find_element(By.CSS_SELECTOR, 'input#btnQry')
search_button.click()
sleep(3)

In [86]:
#切到iframe中，抓page網址
iframe = driver.find_element(By.CSS_SELECTOR, "iframe#iframe-data")
driver.switch_to.frame(iframe)
sleep(2)
#main-section
page_url = driver.find_element(By.CSS_SELECTOR, 'a#hlNext').get_attribute('href')
print(page_url)

https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=2&ot=in


In [87]:
driver.get(page_url) #進去是第二頁，要處理一下

In [88]:
#換頁的話，要對這個剛抓下來的網址做處理，網址會過期的，不能寫死
#https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=2&ot=in

#還是就直接下一頁?用案的不要跌page了

In [89]:
import re

In [90]:
regex01 = "&page=[\d]*"
uuu = "https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=2&ot=in"
test = re.split(regex01,uuu)
test
for i in range(1,26):
    print(f"{test[0]}&page={i}{test[1]}") 

https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=1&ot=in
https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=2&ot=in
https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=3&ot=in
https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=4&ot=in
https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=5&ot=in
https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=6&ot=in
https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=7&ot=in
https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=8&ot=in
https://judgment.judicial.gov.tw/FJUD/qryresultlst.aspx?q=6cce162073a2c1cb1d8644cde16e653d&sort=DS&page=

In [91]:
# 進到page_url之後點取案件
# 其實一樣兩個做法，應該都能在網址失效前解決
# 先抓一頁當中全部的url在一個一個driver.get
# 不然就是用selenium點先試這個好了


In [92]:
# 用selenium
case_url = driver.find_elements(By.CSS_SELECTOR, 'a#hlTitle')
print(len(case_url)) #可以印出有多少筆，那就用這個寫for就好
case_url[3].click() #案件要如何取得全部?
case_html = driver.page_source

20


In [94]:
soup = bs(case_html, "lxml") 

In [95]:
info_list = []
# 案件標題id, 有四個標籤都是使用此選擇器，包含主文....(第四個)，那就抓這個就好了啊...
all = soup.select("div.col-td")
info_list.append({
    "case_id":all[0].get_text().strip(),
    "date":all[1].get_text().strip(),
    "simple_reason":all[2].get_text().strip(),
    "context":all[3].get_text()
})

In [96]:
info_list

[{'case_id': '臺灣高等法院 112 年度上訴字第 4083 號刑事判決',
  'date': '民國 113 年 03 月 05 日',
  'simple_reason': '家暴殺人未遂',
  'context': '\n\n\n\n臺灣高等法院刑事判決112年度上訴字第4083號上\xa0 訴\xa0 人\xa0 臺灣士林地方檢察署檢察官上\xa0 訴\xa0 人\xa0 即\xa0 被\xa0 告\xa0 陳○○選任辯護人\xa0 周政憲律師上列上訴人等因被告家暴殺人未遂案件，不服臺灣士林地方法院111年度訴字第492號，中華民國112年7月28日第一審判決（起訴案號：臺灣士林地方檢察署111年度偵字第19292號），提起上訴，本院判決如下：\xa0 \xa0 主\xa0 文原判決撤銷。陳○○犯殺人未遂罪，處有期徒刑陸年，並應於刑之執行完畢或赦免後，令入相當處所，施以監護肆年。扣案之開山刀壹把及行動電話（iPHONE）壹具沒收。\xa0 \xa0 事\xa0 實一、陳○○與葉○○原為男女朋友，於民國111年4月間分手，為家庭暴力防治法第63條之1第2項所定之親密關係伴侶，但陳○○自97年間起，長期患有思覺失調症之精神疾病，病情發作期間，反覆出現妄想、聽幻覺等症狀，並陸續在國立臺灣大學醫學院附設醫院（下稱臺大醫院）、臺北榮民總醫院、國防醫學院三軍總醫院北投分院（下稱三總北投分院）及天晴診所等醫療院所就醫甚至住院，惟其病識感低落、服藥遵從性差，多次未能定期回診或遵照醫囑定時服藥；而陳○○在與葉○○分手後，因受病情發作後之妄想、聽幻覺等症狀影響，懷疑葉○○以假身分與其交往、葉○○實際為性工作者、其遭葉○○傳染性病等，更認為自己聽到某聲音要自己傷害葉○○，遂於111年8月29日11時許，駕駛車號0000-00號自用小客車，至臺北市○○區○○街與○○路口即葉○○之工作地點附近徘徊，繼於同日13時10分許，在臺北市○○區○○路00號「統一超商經貿門市」前，見葉○○出現在騎樓行走，雖當下陳○○仍明知人之腹部、臉部、頸部均為身體重要部位，且周遭臟器、血管皆屬脆弱之要害，若以利刃朝他人腹部、臉部、頸部攻擊，將可能傷及重要臟器或血管，造成生理機能嚴重受損及大量出血，進而導致死亡結果發生，但因前述精神病症發作，受聽幻覺左右而致其辨識行為違法或依其辨識而行為之能力

In [97]:
# bs4剖完之後要上一頁
driver.back()

上面整個取得單個case再back的過程寫在一個for裡面 (所以這樣應該就可以解決一頁了)

In [98]:
# 再來就是多頁的問題，直接按下一頁，還是一開始取得網址時，就直接拿全部的網址?
# 拿全部的網址可能是比較合理的作法，因為最一開始取得的page_url就是第二頁本來就要處理

In [101]:
# 這邊之後要包try except可能不到25頁
page_url_list = [] 
regex_page_url = "&page=[\d]*"
test = re.split(regex_page_url, page_url)
for each_page in range(1,26):
    page_url_list.append(f"{test[0]}&page={each_page}{test[1]}") 


In [103]:
# 迭所有page
count = 0
for page in page_url_list:
    driver.get(page)
    count+=1
    if count > 10:
        break